In [1]:
import findspark
findspark.init()

In [2]:
import happybase
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("zad") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/01/05 22:20:21 WARN util.Utils: Your hostname, node1 resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
24/01/05 22:20:21 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/05 22:20:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/05 22:20:23 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
VM_adress = 'localhost'
connection = happybase.Connection(VM_adress, timeout=999999)


In [4]:
all = spark.read.parquet('/user/projekt/gold/flights_weather/')
all.show(10)

+------------+-------------+-----------+------------+-------------+-------------+---------------+---------------------+------------+------------+--------------------+--------------+---------------------+---------------+---------------+-------------+-------------------+------------+------------+-------------------+----------------+----------------+-----------------------+-----------------+---------------+-------------------+--------------+--------------+--------------+-------------------+------------------+------------------+------------------------------+------------------------------+------------------------------+-----------------------------+-----------------------------+-------------------------------+-----------+-----------+-------------+-----------+-------------+-------------+--------------+--------------+-------------+--------------+---------------------+-------------------+-------------------+--------------------+-------------------+----+-----+---+--------------+---------------

In [5]:
structure = {
    'flight': [
        'arrival_year',
        'arrival_month',
        'arrival_day',
        'arrival_iata',
        'aircraft_iata',
        'aircraft_icao',
        'aircraft_icao24',
        'aircraft_registration',
        'airline_iata',
        'airline_icao',
        'airline_name',
        'arrival_actual',
        'arrival_actual_runway',
        'arrival_airport',
        'arrival_baggage',
        'arrival_delay',
        'arrival_estimated',
        'arrival_gate',
        'arrival_icao',
        'arrival_scheduled',
        'arrival_terminal',
        'arrival_timezone',
        'departure_actual_runway',
        'departure_airport',
        'departure_delay',
        'departure_estimated',
        'departure_gate',
        'departure_iata',
        'departure_icao',
        'departure_scheduled',
        'departure_terminal',
        'departure_timezone',
        'flight_codeshared_airline_iata',
        'flight_codeshared_airline_icao',
        'flight_codeshared_airline_name',
        'flight_codeshared_flight_iata',
        'flight_codeshared_flight_icao',
        'flight_codeshared_flight_number',
        'flight_iata',
        'flight_icao',
        'flight_number',
        'flight_date',
        'flight_status'
    ],

    'weather': [
        'arrival_to_poland',
        'arrival_station_name',
        'arrival_station_code',
        'arrival_temperature_avg',
        'arrival_temperature_avg_status',
        'arrival_humidity_avg',
        'arrival_humidity_avg_status',
        'arrival_wind_speed_avg',
        'arrival_wind_speed_avg_status',
        'arrival_cloud_coverage_avg',
        'arrival_cloud_coverage_avg_status',
        'arrival_temperature_max',
        'arrival_temperature_max_status',
        'arrival_temperature_min',
        'arrival_temperature_min_status',
        'arrival_temperature_ground_min',
        'arrival_temperature_ground_min_status',
        'arrival_rain',
        'arrival_rain_status',
        'arrival_rain_type',
        'arrival_snow_height',
        'arrival_snow_height_status',
        'departure_station_name',
        'departure_station_code',
        'departure_temperature_avg',
        'departure_temperature_avg_status',
        'departure_humidity_avg',
        'departure_humidity_avg_status',
        'departure_wind_speed_avg',
        'departure_wind_speed_avg_status',
        'departure_cloud_coverage_avg',
        'departure_cloud_coverage_avg_status',
        'departure_temperature_max',
        'departure_temperature_max_status',
        'departure_temperature_min',
        'departure_temperature_min_status',
        'departure_temperature_ground_min',
        'departure_temperature_ground_min_status',
        'departure_rain',
        'departure_rain_status',
        'departure_rain_type',
        'departure_snow_height',
        'departure_snow_height_status'
    ]
}

In [6]:
small_df = all

In [7]:
columns_selection = []
for key, value in structure.items():
    columns_selection += value

small_df = small_df[columns_selection]

In [8]:
column_families = []
for key, value in structure.items():
    column_families += [key]
column_families

['flight', 'weather']

In [27]:
table_name = 'projekt'
connection = happybase.Connection(VM_adress, timeout=999999)

# jesli istnieje to usunmy
if table_name.encode() in connection.tables():
    print(f"tabela {table_name} juz jest, wiec usuwamy")
    connection.disable_table(table_name)
    connection.delete_table(table_name, disable = True)

# stworz tabelelsit
table = connection.create_table(
    table_name,
    {
        cf: dict(MAX_VERSIONS=10) for cf in column_families
    })
print(f"tabela {table_name} utworzona")



tabela projekt juz jest, wiec usuwamy
tabela projekt utworzona


In [10]:
def encode_if_not_none(value):
    return str(value).encode('utf-8') if value is not None else None

In [11]:
def return_cf_for_col(col, structure):
    for key, value in structure.items():
        if col in value:
            return key
    return None
    

In [12]:
def connect():
    VM_adress = 'localhost'
    connection = happybase.Connection(VM_adress, timeout=None)
    table = connection.table(table_name)
    return table

In [13]:
from pyspark.sql.functions import col
for column in small_df.columns:
    small_df = small_df.withColumn(column, col(column).cast("string"))

In [14]:
small_df = small_df.toPandas()

In [15]:
table = connect()


for i in range (len(small_df)):
    flight_icao = small_df.loc[i, 'flight_icao']
    departure_scheduled = small_df.loc[i, 'departure_scheduled']
    row_key = f'row{flight_icao}_{departure_scheduled}'
    data = {}
    columns_not_null = list(small_df.columns[small_df.iloc[i].notna()])

    for col in columns_selection:
        if col in columns_not_null:
            family = return_cf_for_col(col, structure)
            data[f'{family}:{col}'.encode()] = encode_if_not_none(small_df.loc[i, col])

    table.put(row_key, data)


    

In [16]:
def print_table(limit = None):
    table = connect()
    
    for key, data in table.scan(None):
        print(f'KEY: {key}')
        for column, value in data.items():
            print(f'\tCOLUMNS: {column} VALUE: {value}')

In [31]:
print_table()

KEY: b'rowAFR1147_2023-12-31 13:05:00'
	COLUMNS: b'flight:aircraft_iata' VALUE: b'BCS3'
	COLUMNS: b'flight:aircraft_icao' VALUE: b'BCS3'
	COLUMNS: b'flight:aircraft_icao24' VALUE: b'39E686'
	COLUMNS: b'flight:aircraft_registration' VALUE: b'F-HZUG'
	COLUMNS: b'flight:airline_iata' VALUE: b'AF'
	COLUMNS: b'flight:airline_icao' VALUE: b'AFR'
	COLUMNS: b'flight:airline_name' VALUE: b'Air France'
	COLUMNS: b'flight:arrival_actual_runway' VALUE: b'2023-12-31T13:15:00+00:00'
	COLUMNS: b'flight:arrival_airport' VALUE: b'Frederic Chopin'
	COLUMNS: b'flight:arrival_day' VALUE: b'31'
	COLUMNS: b'flight:arrival_delay' VALUE: b'10'
	COLUMNS: b'flight:arrival_estimated' VALUE: b'2023-12-31 15:35:00'
	COLUMNS: b'flight:arrival_iata' VALUE: b'CDG'
	COLUMNS: b'flight:arrival_icao' VALUE: b'LFPG'
	COLUMNS: b'flight:arrival_month' VALUE: b'12'
	COLUMNS: b'flight:arrival_scheduled' VALUE: b'2023-12-31 15:35:00'
	COLUMNS: b'flight:arrival_terminal' VALUE: b'2F'
	COLUMNS: b'flight:arrival_timezone' VALUE: 

In [18]:
table

<happybase.table.Table name=b'projekt'>